In [6]:
import pandas as pd
import numpy as np
import statistics
import math
import os

In [7]:
# Set working directory
os.chdir('C:/Users/sniderc/Desktop/CIS5500/Project/')

# Load Data

In [9]:
beneficiary_test = pd.read_csv('Test_Beneficiarydata-1542969243754.csv')
beneficiary_train = pd.read_csv('Train_Beneficiarydata-1542865627584.csv')

beneficiary = beneficiary_test.append(beneficiary_train)

inpatient_test = pd.read_csv('Test_Inpatientdata-1542969243754.csv')
inpatient_train = pd.read_csv('Train_Inpatientdata-1542865627584.csv')

# Combine test and training sets into one table
inpatient = inpatient_test.append(inpatient_train)

outpatient_test = pd.read_csv('Test_Outpatientdata-1542969243754.csv')
outpatient_train = pd.read_csv('Train_Outpatientdata-1542865627584.csv')

# Combine test and training sets into one table
outpatient = outpatient_test.append(outpatient_train)

provider = pd.read_csv('Train-1542865627584.csv')


C:\Users\sniderc\AppData\Local\Temp\ipykernel_222648\3460186906.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  beneficiary = beneficiary_test.append(beneficiary_train)
C:\Users\sniderc\AppData\Local\Temp\ipykernel_222648\3460186906.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inpatient = inpatient_test.append(inpatient_train)
C:\Users\sniderc\AppData\Local\Temp\ipykernel_222648\3460186906.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outpatient = outpatient_test.append(outpatient_train)


In [10]:
# Read in additional data sources (diagnosis and procedure files from CMS)
diagnosis = pd.read_excel('Diagnosis.xlsx')

procedure = pd.read_excel('Procedure.xlsx')

# Patient Table

In [136]:
# Remove duplicate entries (from combining training and test datasets)
beneficiary.drop_duplicates(keep='first', inplace=True)

# Check for Unique Primary Key
print("Number of Rows: " + str(len(beneficiary)))
print("Unique Patients: " + str(beneficiary.BeneID.nunique()))

Number of Rows: 148072
Unique Patients: 148072


In [137]:
beneficiary[['BeneID','DOB','DOD','Gender','Race','State','County']].to_csv('Patient.csv',sep=',',index=False)

# Comorbidity

In [140]:
# Reformat from wide to long

comorbidity_long = pd.melt(beneficiary, id_vars='BeneID',
        value_vars=['ChronicCond_Alzheimer', 'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
                    'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression', 'ChronicCond_Diabetes', 
                    'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',],
       var_name='ChronicCondition', value_name='ConditionValue')

comorbidity_long['Comorbidity'] = np.where((comorbidity_long.ChronicCondition=='ChronicCond_Alzheimer') & 
                                         (comorbidity_long.ConditionValue==1), 'Alzheimers Disease', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_Heartfailure') & 
                                         (comorbidity_long.ConditionValue==1), 'Heart Failure', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_KidneyDisease') & 
                                         (comorbidity_long.ConditionValue==1), 'Chronic Kidney Disease', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_KidneyDisease') & 
                                         (comorbidity_long.ConditionValue==1), 'Chronic Kidney Disease', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_Cancer') & 
                                         (comorbidity_long.ConditionValue==1), 'Cancer', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_ObstrPulmonary') & 
                                         (comorbidity_long.ConditionValue==1), 'Chronic Obstructive Pulmonary Disease', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_Depression') & 
                                         (comorbidity_long.ConditionValue==1), 'Depression', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_Diabetes') & 
                                         (comorbidity_long.ConditionValue==1), 'Diabetes', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_IschemicHeart') & 
                                         (comorbidity_long.ConditionValue==1), 'Ischemic Heart Disease', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_Osteoporasis') & 
                                         (comorbidity_long.ConditionValue==1), 'Osteoporosis', \
                                np.where((comorbidity_long.ChronicCondition=='ChronicCond_rheumatoidarthritis') & 
                                         (comorbidity_long.ConditionValue==1), 'Rheumatoid Arthritis', \
                                         'None')))))))))))

comorbidity = comorbidity_long[comorbidity_long.Comorbidity!='None'][['BeneID','Comorbidity']]

comorbidity.groupby('Comorbidity').size()

Comorbidity
Alzheimers Disease                       48298
Cancer                                   17383
Chronic Kidney Disease                   45020
Chronic Obstructive Pulmonary Disease    34145
Depression                               51596
Diabetes                                 87663
Heart Failure                            71892
Ischemic Heart Disease                   98811
Osteoporosis                             40050
Rheumatoid Arthritis                     37214
dtype: int64

In [144]:
# Check for Unique Primary Key
print("Number of Rows: " + str(len(comorbidity)))
print("Unique Patient-Comorbidities: " + str(len(comorbidity[['BeneID','Comorbidity']].drop_duplicates())))

Number of Rows: 532072
Unique Patient-Comorbidities: 532072


In [145]:
comorbidity.to_csv('Comorbidity.csv',sep=',',index=False)

# Provider

In [147]:
# Rename primary key
provider.rename(columns={'Provider':'ProviderID'}, inplace=True)
provider.head()

,ProviderID,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [174]:
# Add providers from Encounters who were not included in the Provider Fraud file
encounter_providers = pd.DataFrame(outpatient.Provider.append(inpatient.Provider).unique(), columns=['ProviderID'])

encounter_providers['PotentialFraud'] = 'Unknown'

provider = provider.append(encounter_providers).drop_duplicates(subset=['ProviderID'], keep='first')


C:\Users\sniderc\AppData\Local\Temp\ipykernel_222648\2563541899.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  encounter_providers = pd.DataFrame(outpatient.Provider.append(inpatient.Provider).unique(), columns=['ProviderID'])
C:\Users\sniderc\AppData\Local\Temp\ipykernel_222648\2563541899.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  provider = provider.append(encounter_providers).drop_duplicates(subset=['ProviderID'], keep='first')


In [175]:
# Check for Unique Primary Key
print("Number of Rows: " + str(len(provider)))
print("Unique Providers: " + str(provider.ProviderID.nunique()))

Number of Rows: 6763
Unique Providers: 6763


In [177]:
provider.to_csv('Provider.csv',sep=',',index=False)

# Encounter

In [93]:
outpatient.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode'],
      dtype='object')

In [101]:
# Set Type to "Outpatient"
outpatient['Type'] = 'Outpatient'

# Set Type to "Inpatient"
inpatient['Type'] = 'Inpatient'

encounter = outpatient[['BeneID', 'ClaimID', 'Type', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
            'InscClaimAmtReimbursed', 'DeductibleAmtPaid']].append(inpatient[['BeneID', 'ClaimID', 'Type', \
                                                                              'ClaimStartDt', 'ClaimEndDt', 'Provider',
                                                                              'InscClaimAmtReimbursed', 'DeductibleAmtPaid']])

encounter.rename(columns={'ClaimStartDt': 'ClaimStartDate', 'ClaimEndDt': 'ClaimEndDate', 'Provider': 'ProviderID',
                         'InscClaimAmtReimbursed': 'AmountReimbursed', 'DeductibleAmtPaid': 'Deductible'}, inplace=True)

encounter.head()

C:\Users\sniderc\AppData\Local\Temp\ipykernel_222648\403598109.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  encounter = outpatient[['BeneID', 'ClaimID', 'Type', 'ClaimStartDt', 'ClaimEndDt', 'Provider',


,BeneID,ClaimID,Type,ClaimStartDate,ClaimEndDate,ProviderID,AmountReimbursed,Deductible
0,BENE11001,CLM392397,Outpatient,2009-06-02,2009-06-02,PRV55962,30,0.0
1,BENE11001,CLM430760,Outpatient,2009-06-23,2009-06-23,PRV56112,30,0.0
2,BENE11007,CLM233081,Outpatient,2009-03-07,2009-03-07,PRV56979,200,0.0
3,BENE11007,CLM496381,Outpatient,2009-07-29,2009-07-29,PRV56573,10,0.0
4,BENE11007,CLM521391,Outpatient,2009-08-12,2009-08-12,PRV56573,10,0.0


In [150]:
# Check for Unique Primary Key
print("Number of Rows: " + str(len(encounter)))
print("Unique Encounters: " + str(encounter.ClaimID.nunique()))

Number of Rows: 693603
Unique Encounters: 693603


In [151]:
encounter.to_csv('Encounter.csv',sep=',',index=False)

In [129]:
inpatient.rename(columns={'AdmissionDt': 'AdmissionDate', 'DischargeDt': 'DischargeDate'}, inplace=True)

In [152]:
# Check for Unique Primary Key
print("Number of Rows: " + str(len(inpatient)))
print("Unique Encounters: " + str(inpatient.ClaimID.nunique()))

Number of Rows: 50025
Unique Encounters: 50025


In [155]:
inpatient[['ClaimID','Type','AdmissionDate','DischargeDate']].to_csv('Inpatient.csv',sep=',',index=False)

In [153]:
# Check for Unique Primary Key
print("Number of Rows: " + str(len(outpatient)))
print("Unique Encounters: " + str(outpatient.ClaimID.nunique()))

Number of Rows: 643578
Unique Encounters: 643578


In [154]:
outpatient[['ClaimID','Type']].to_csv('Outpatient.csv',sep=',',index=False)

# Diagnosis

In [156]:
# Check for Unique Primary Key
print("Number of Rows: " + str(len(diagnosis)))
print("Unique Diagnoses: " + str(diagnosis.DxCode.nunique()))

Number of Rows: 14653
Unique Diagnoses: 14653


In [157]:
diagnosis.to_csv('Diagnosis.csv',sep=',',index=False)

In [125]:
# Combine outpatient and inpatient diagnoses
encounter_diagnosis_wide = outpatient[['ClaimID','ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 
                                       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5','ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 
                                       'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10']]\
.append(inpatient[['ClaimID','ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10']])

# Transform the data 
encounter_diagnosis_long = pd.melt(encounter_diagnosis_wide, id_vars='ClaimID',
        value_vars=['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 
                    'ClmDiagnosisCode_5','ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 
                    'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10'],
       var_name='DxNum', value_name='DxCode')

# Create the 'DxNum' variable
encounter_diagnosis_long['DxNum'] = encounter_diagnosis_long['DxNum'].apply(lambda x: x.replace("ClmDiagnosisCode_", ""))

# Remove rows where diagnosis was not populated
encounter_diagnosis = encounter_diagnosis_long[~encounter_diagnosis_long.DxCode.isna()]

encounter_diagnosis.head()

C:\Users\sniderc\AppData\Local\Temp\ipykernel_222648\2751602408.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  encounter_diagnosis_wide = outpatient[['ClaimID','ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',


,ClaimID,DxNum,DxCode
0,CLM392397,1,V5832
1,CLM430760,1,9594
2,CLM233081,1,7248
3,CLM496381,1,58889
4,CLM521391,1,V666


In [162]:
# Check for Unique Primary Key
print("Number of Rows: " + str(len(encounter_diagnosis)))
print("Unique Encounter Diagnoses: " + str(len(encounter_diagnosis[['ClaimID','DxNum']].drop_duplicates())))

Number of Rows: 2085893
Unique Encounter Diagnoses: 2085893


In [163]:
encounter_diagnosis.to_csv('EncounterDiagnosis.csv',sep=',',index=False)

# Procedure

In [160]:
# Drop duplicate procedures 
procedure.drop_duplicates(subset=['ProcedureCode'], keep='first', inplace=True)

# Check for Unique Primary Key
print("Number of Rows: " + str(len(procedure)))
print("Unique Procedures: " + str(procedure.ProcedureCode.nunique()))

Number of Rows: 3809
Unique Procedures: 3809


In [161]:
procedure.to_csv('Procedure.csv',sep=',',index=False)

In [127]:
# Combine outpatient and inpatient diagnoses
encounter_procedure_wide = outpatient[['ClaimID','ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3', 
                                       'ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6']]\
.append(inpatient[['ClaimID','ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4', 
                   'ClmProcedureCode_5', 'ClmProcedureCode_6']])

# Transform the data 
encounter_procedure_long = pd.melt(encounter_procedure_wide, id_vars='ClaimID',
        value_vars=['ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4', 
                    'ClmProcedureCode_5', 'ClmProcedureCode_6'],
       var_name='ProcedureNum', value_name='ProcedureCode')

# Create the 'DxNum' variable
encounter_procedure_long['ProcedureNum'] = encounter_procedure_long['ProcedureNum'].apply(lambda x: x.replace("ClmProcedureCode_", ""))

# Remove rows where procedure wasn't populated
encounter_procedure = encounter_procedure_long[~encounter_procedure_long.ProcedureCode.isna()]

encounter_procedure.head()

C:\Users\sniderc\AppData\Local\Temp\ipykernel_222648\681076188.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  encounter_procedure_wide = outpatient[['ClaimID','ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3',


,ClaimID,ProcedureNum,ProcedureCode
6864,CLM203794,1,9672.0
9238,CLM127239,1,5221.0
16246,CLM112459,1,7935.0
18072,CLM82005,1,9671.0
29037,CLM350592,1,3772.0


In [164]:
# Check for Unique Primary Key
print("Number of Rows: " + str(len(encounter_procedure)))
print("Unique Encounter Procedures: " + str(len(encounter_procedure[['ClaimID','ProcedureNum']].drop_duplicates())))

Number of Rows: 36882
Unique Encounter Procedures: 36882


In [165]:
encounter_procedure.to_csv('EncounterProcedure.csv',sep=',',index=False)